In [1]:
import yfinance as yf
from datetime import datetime
from time import time
import stock
from importlib import reload
import concurrent.futures

In [2]:
stock_symbols = [
    "MMM",
    "AOS",
    "ABT",
    "ABBV",
    "ABMD",
    "ACN",
    "ATVI",
    "ADM",
    "ADBE",
    "ADP",
    "AAP",
    "AES",
    "AFL",
    "A",
    "APD",
    "AKAM",
    "ALK",
    "ALB",
    "ARE",
    "ALGN",
    "ALLE",
    "LNT",
    "ALL",
    "GOOGL",
    "GOOG",
    "MO",
    "AMZN",
    "AMCR",
    "AMD",
    "AEE",
    "AAL",
    "AEP",
    "AXP",
    "AIG",
    "AMT",
    "AWK",
    "AMP",
    "ABC",
    "AME",
    "AMGN",
    "APH",
    "ADI",
    "ANSS",
    "AON",
    "APA",
    "AAPL",
    "AMAT",
    "APTV",
    "ANET",
    "AJG",
    "AIZ",
    "T",
    "ATO",
    "ADSK",
    "AZO",
    "AVB",
    "AVY",
    "BKR",
    "BALL",
    "BAC",
    "BBWI",
    "BAX",
    "BDX",
    "WRB",
    "BRK-B",
    "BBY",
    "BIO",
    "TECH",
    "BIIB",
    "BLK",
    "BK",
    "BA",
    "BKNG",
    "BWA",
    "BXP",
    "BSX",
]


In [42]:
reload(stock)

start_time = time()
stocks = []
for symbol in stock_symbols:
    stocks.append(stock.Stock(symbol))
print("Elapsed time: {:.3f}s".format(time() - start_time))


Elapsed time: 89.763s


In [8]:
stocks = []


def create_stock_from_symbol(symbol):
    return stock.Stock(symbol)


start_time = time()
with concurrent.futures.ProcessPoolExecutor(max_workers=61) as executor:
    future_proc = [
        executor.submit(create_stock_from_symbol, symbol) for symbol in stock_symbols
    ]
    for future in concurrent.futures.as_completed(future_proc):
        stocks.append(future.result())
print("Elapsed time: {:.3f}s".format(time() - start_time))


Elapsed time: 1.742s


In [17]:
from indicator import EMA, MACD, RSI, CipherB, Indicator, StochRSI

reload(stock)

rsi = RSI()
stochrsi = StochRSI()
macd = MACD()
ema = EMA()
cipher_b = CipherB()
indicators = [rsi, stochrsi, macd, ema, cipher_b]

stocks = [stock.Stock("AAPL")]

def add_indicators(stock, indicators):
    stock.add_indicators(indicators)
    return stock

with concurrent.futures.ProcessPoolExecutor(61) as executor:
    future_proc = [
        executor.submit(add_indicators, s, indicators) for s in stocks
    ]
    for future in concurrent.futures.as_completed(future_proc):
        stocks.append(future.result())
print(len(stocks[-1].klines.columns))

adding
26


In [11]:
import tweepy

api_key = r"uZeYmTkM6q9VDYvMYDLialyNN"
api_secret = r"zQbXWRFMjaRzLW7sGxrYPjd0ewxyEU1g0q30X7EHYw9LtUiX8z"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAG8AhQEAAAAA4EckVEMHWEyM3rfxgNyiFTCy5hg%3DgSiAHWLHyAKtt9erA7l3dDMCgnQgoYifatBNcoUNrJUSh24D4w"
access_token = r"4010463713-SthnZ0YtItfZIE4hRfy53TNKlUbTGBiXgdvpdiK"
access_token_secret = r"HAUpQeulLIA00MiK39FwolXlRSq41ackVTzBs1iuibD10"


# Authenticate to Twitter
client = tweepy.Client(bearer_token)


query = "stocks #CMG lang:en"
tweet = client.search_recent_tweets(
    query=query,
    max_results=100,
    tweet_fields=[
        "id",
        "public_metrics"
    ],
)
tweet

In [61]:
import yfinance as yf

yf.Ticker("MSFT").stats()

{'defaultKeyStatistics': {'annualHoldingsTurnover': None,
  'enterpriseToRevenue': 9.073,
  'beta3Year': None,
  'profitMargins': 0.36686,
  'enterpriseToEbitda': 18.359,
  '52WeekChange': -0.16839957,
  'morningStarRiskRating': None,
  'forwardEps': 11.95,
  'revenueQuarterlyGrowth': None,
  'sharesOutstanding': 7457889792,
  'fundInceptionDate': None,
  'annualReportExpenseRatio': None,
  'totalAssets': None,
  'bookValue': 22.313,
  'sharesShort': 40544332,
  'sharesPercentSharesOut': 0.0054,
  'fundFamily': None,
  'lastFiscalYearEnd': 1656547200,
  'heldPercentInstitutions': 0.72058,
  'netIncomeToCommon': 72737996800,
  'trailingEps': 9.65,
  'lastDividendValue': 0.62,
  'SandP52WeekChange': -0.11115879,
  'priceToBook': 10.968494,
  'heldPercentInsiders': 0.00077,
  'nextFiscalYearEnd': 1719705600,
  'yield': None,
  'mostRecentQuarter': 1656547200,
  'shortRatio': 1.95,
  'sharesShortPreviousMonthDate': 1659052800,
  'floatShares': 7451254348,
  'beta': 0.942868,
  'enterpriseV

In [56]:
class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = "https://publish.twitter.com/oembed?url={}".format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
            self.text = self.text.replace('class="twitter-tweet"', 'class="twitter-tweet" width="325"')
        else:
            self.text = s

    def _repr_html_(self):
        return self.text

    def component(self):
        return components.html(self.text, height=200, width=23)


url = "https://twitter.com/twitter/statuses/{}".format(tweet.id)
t = Tweet(url.format(tweet_id=tweet.id)).text
t

'<blockquote class="twitter-tweet" width="325"><p lang="en" dir="ltr"><a href="https://twitter.com/search?q=%24AAPL&amp;src=ctag&amp;ref_src=twsrc%5Etfw">$AAPL</a> <a href="https://twitter.com/hashtag/AAPL?src=hash&amp;ref_src=twsrc%5Etfw">#AAPL</a> Breaking down through the volume shelf and VWAP zone, bull setup invalidated with a daily close below this area, which is looking likely. <br><br>Not a ton of liquidity below until $144-$145. <a href="https://t.co/qrX9rJKrnA">pic.twitter.com/qrX9rJKrnA</a></p>&mdash; Jake Wujastyk (@Jake__Wujastyk) <a href="https://twitter.com/Jake__Wujastyk/status/1570428289612935178?ref_src=twsrc%5Etfw">September 15, 2022</a></blockquote>\n<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>\n'